In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import tqdm.notebook as tq

# Adding book title to each review

In [3]:
rev = pd.read_pickle('csc2515-project-main/csc2515-project-main/data/goodreads.pkl')
books = pd.read_csv('csc2515-project-main/csc2515-project-main/data/goodreads_book_info.csv')

rev.shape, books.shape

((275607, 7), (2360655, 3))

In [4]:
rev.head()

,user_id,timestamp,review_sentences,rating,has_spoiler,book_id,review_id
0,5b78fd996dd2af3e863bec412a410512,2012-01-02,"[[0, Essential reading for anyone serious abou...",5,False,29999,ad2bf9e7e72997ec053fe9d666a76240
1,e21627c07b1c16a64b1d55afb0801cd3,2015-12-22,"[[0, Very informative.], [0, I feel like this ...",3,False,6882274,a0ce4c9e8a6e644d90851330494615e4
2,40272245ceb3404a973ab053f636dcf8,2016-02-29,"[[0, Really 4.4 stars - but goodreads only let...",4,False,17565845,78c8562226c80c7b7ca89afffa2c17dc
3,788c16647e9ba6f337a76f51b92cadc9,2013-08-23,"[[0, This was on my wish list a long time ago,...",1,False,9681214,6e7a6c60e7cdca91430dfb6113ad50b3
4,60ba13d8c742b84b2ec2445d9e04afe6,2013-11-23,"[[0, I loved this book!!], [0, When I first he...",5,True,11614718,2abcd2f0c63ea8de00024f7f22e167d4


In [34]:
books.head()

,description,book_id,title,reviews
115,The future world is at peace.\nElla Shepherd h...,22642971,The Body Electric,Oooh this is coming on my birthday. PERFECT BI...
282,Ready or not...love will find a way \nSingle d...,32336119,Worth the Wait (Guthrie Brothers #2),I love small town romances. A place where ever...
367,This is Maddy Turner's lucky day. The civilize...,2741853,Slow Hands,"Cute story, if a little predictable. For a FRE..."
376,A classic work of science fiction by renowned ...,12077902,Solaris: The Definitive Edition,Having been confused by both movie versions of...
383,After a series of explosive encounters with tw...,7843586,"More (More, #1)",A new favorite author! This is going into my f...


In [12]:
rev.book_id = rev.book_id.apply(np.int64)

In [16]:
books = books[books.book_id.isin(rev.book_id)]

In [17]:
books.shape

(24873, 3)

In [18]:
rev = rev.merge(books[['book_id', 'title']], on='book_id')

In [22]:
rev.to_pickle('csc2515-project-main/csc2515-project-main/data/goodreads.pkl')

In [33]:
rev['flat_reviews'] = rev.review_sentences.apply(lambda x: ' '.join([s[1] for s in x]))
books['reviews'] = books.book_id.apply(lambda x: ' '.join(rev[rev.book_id == x].flat_reviews.values))

C:\Users\wojtek\AppData\Local\Temp/ipykernel_11096/3508225722.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books['reviews'] = books.book_id.apply(lambda x: ' '.join(rev[rev.book_id == x].flat_reviews.values))


In [37]:
books.reviews.values[3]

"Having been confused by both movie versions of this book I decided to give the original a go and see if it's any better. Before I started on this I imagining it was going to be a total mess and I might have to abandon it. But, it was the exact opposite! The book quickly gets to the heart of the matter and all the confusion of the movies was explained in the first 10-20%. Having now read Solaris, I seriously cannot see how this could be made into an effective movie without tons of voice overs. Most of the drama is psychological in nature and revolves around the central characters thoughts and emotions. What both the movies kind of left out is the nature of the planet Solaris. It contains a vast alien intelligence, totally unlike any thing encountered before and mostly incomprehensible to the scientists trying to study it. It's a great example of alien contact and world building rolled into one. Anyone who didn't like the movie, but is still wondering why this is such a renowned work sh

In [39]:
books.to_csv('csc2515-project-main/csc2515-project-main/data/goodreads_books_revs.csv', index=False)

# building tf-idf matrix

In [48]:
count_vect = CountVectorizer()
word_idx = count_vect.fit_transform(books.reviews.values)
word_idx.shape

(24873, 168886)

In [50]:
count_vect.vocabulary_.get('snout')

137821

In [52]:
tfidf_transformer = TfidfTransformer().fit(word_idx)
books_tfidf = tfidf_transformer.transform(word_idx)
books_tfidf.shape

(24873, 168886)

In [54]:
books_tfidf

<24873x168886 sparse matrix of type '<class 'numpy.float64'>'
	with 14092982 stored elements in Compressed Sparse Row format>

In [87]:
df = pd.read_pickle('csc2515-project-main/csc2515-project-main/data/goodreads_sent_spoil.pkl')

In [88]:
df.head()

,sentence,has_spoiler
0,Essential reading for anyone serious about mys...,0
1,Hammett is the grandfather of the hard-bitten ...,0
2,While his Continental Op was the prototype for...,0
3,"Expect sharp dialog, vivid characters and a vo...",0
4,If you've seen John Huston's 1941 film version...,0


In [89]:
# df = df.sample(frac=0.1)

In [90]:
df_train, df_test = train_test_split(
  df,
  test_size=0.2,
  random_state=42
)

In [79]:
X_train = tfidf_transformer.transform(count_vect.transform(df_train.sentence.values))

# Fitting NB classifier

In [80]:
nb = MultinomialNB().fit(X_train, df_train.has_spoiler)

In [91]:
X_test = tfidf_transformer.transform(count_vect.transform(df_test.sentence.values))
predicted = nb.predict(X_test)

In [92]:
roc_auc_score(df_test.has_spoiler, predicted)

0.7153545101231146

In [83]:
predicted[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

# that didn't go to well! Let's try with a balanced dataset

In [85]:
df = pd.concat([df[df.has_spoiler==1], df[df.has_spoiler==0].sample(len(df[df.has_spoiler==1]))]).sample(frac=1)
df_train, df_test = train_test_split(
  df,
  test_size=0.2,
  random_state=42
)

X_train = tfidf_transformer.transform(count_vect.transform(df_train.sentence.values))
X_test = tfidf_transformer.transform(count_vect.transform(df_test.sentence.values))

nb = MultinomialNB().fit(X_train, df_train.has_spoiler)
predicted = nb.predict(X_test)

In [86]:
roc_auc_score(df_test.has_spoiler, predicted)

0.6870193939592415

# Now that's much better, but somehow doesn't cut it
Looks like the task is a wee bit too tough for a simple model like this, but the transformer-based models might benefit from these features, or at least book titles

In [93]:
# roc-auc: 69% for a balanced test set, 72% for the original proportions set

# Preparing transformer input sentences: adding book titles

In [96]:
rev = pd.read_pickle('csc2515-project-main/csc2515-project-main/data/goodreads.pkl')
sent = []
spoil = []
for i in tq.tqdm(range(len(rev))):
    for j in range(len(rev.iloc[i]['review_sentences'])):
        sent.append(rev.iloc[i].title + ' [SEP] ' + rev.iloc[i]['review_sentences'][j][1])   
        spoil.append(rev.iloc[i]['review_sentences'][j][0])

  0%|          | 0/275607 [00:00<?, ?it/s]

In [97]:
df = pd.DataFrame(list(zip(sent, spoil)), 
               columns =['sentence', 'has_spoiler']) 

In [99]:
df.to_pickle("csc2515-project-main/csc2515-project-main/data/goodreads_sent_spoil_titles.pkl")

In [100]:
df.head()

,sentence,has_spoiler
0,The Maltese Falcon [SEP] Essential reading for...,0
1,The Maltese Falcon [SEP] Hammett is the grandf...,0
2,The Maltese Falcon [SEP] While his Continental...,0
3,"The Maltese Falcon [SEP] Expect sharp dialog, ...",0
4,The Maltese Falcon [SEP] If you've seen John H...,0


In [3]:
rev = pd.read_pickle('data/flattened_15K_test.pkl')
rev.head()

,user_id,timestamp,book_id,book_title,sent_num,sent_spoil,sentence,rating,has_spoiler,review_id
111980,bdca7a98aaf41a86e637c63ae39660e1,2015-06-13,22308717,The Last Time We Say Goodbye,9,0,I envy Alexis Riggs for that.,4,False,8f776a27a80e922e4a7d8cd493a41959
173287,c800ba07d8d3d7c81f122cffcc78bc73,2014-06-09,11713449,"Delirium (Delirium, #1)",19,0,Lena also had some fairly solid relationships ...,4,False,20b8a1c85ea93aa32ab404e2820ea80e
52708,24434c68039ba698f1ae91ecb7065f2a,2016-05-12,16034235,"Throne of Glass (Throne of Glass, #1)",8,0,It is believed that magic no longer exists.,4,False,87c6f9a4bd270ba942b74632d3227093
93324,5ffbda5c69a7ed61e185b9878b0d622d,2012-08-13,11919,"Haunted (Women of the Otherworld, #5)",29,0,They simply click.,3,False,c68a5b3433ee9da0021c02373084b497
71643,b012ddad3df33ae31b180d0b70095b42,2014-06-27,30122,The Missing Piece,3,0,Will the circle ever miss that piece it found ...,4,False,66b061efe7e6057792f4042f97712c12


In [5]:
sent = []
spoil = []
for i in tq.tqdm(range(len(rev))):
    sent.append(rev.iloc[i].book_title + ' [SEP] ' + rev.iloc[i].sentence)   
    spoil.append(rev.iloc[i].sent_spoil)

df = pd.DataFrame(list(zip(sent, spoil)), 
               columns =['sentence', 'has_spoiler']) 

  0%|          | 0/15001 [00:00<?, ?it/s]

In [7]:
df.has_spoiler.value_counts()

0    14485
1      516
Name: has_spoiler, dtype: int64

In [8]:
df.to_pickle("data/15ktest_goodreads_sent_spoil_titles.pkl")

In [4]:
df = pd.read_pickle("data/goodreads_sent_spoil_titles.pkl")
df.shape

(3534334, 2)

In [5]:
test = pd.read_pickle('data/15ktest_goodreads_sent_spoil_titles.pkl')
test.shape

(15001, 2)

In [6]:
test.head()

,sentence,has_spoiler
0,The Last Time We Say Goodbye [SEP] I envy Alex...,0
1,"Delirium (Delirium, #1) [SEP] Lena also had so...",0
2,"Throne of Glass (Throne of Glass, #1) [SEP] It...",0
3,"Haunted (Women of the Otherworld, #5) [SEP] Th...",0
4,The Missing Piece [SEP] Will the circle ever m...,0


In [8]:
# making sure test and train/val sets are disjoint
df = df[~df.sentence.isin(test.sentence)]
df.shape

(3518910, 2)

In [10]:
df.to_pickle("data/goodreads_sent_spoil_titles.pkl")

In [11]:
# saving a balanced version of the dataset
df = pd.concat([df[df.has_spoiler==1], df[df.has_spoiler==0].sample(len(df[df.has_spoiler==1]))]).sample(frac=1)
df.shape

(228460, 2)

In [12]:
df.to_pickle("data/goodreads_sent_spoil_titles_balanced.pkl")

In [13]:
df.has_spoiler.value_counts()

0    114230
1    114230
Name: has_spoiler, dtype: int64